In [319]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [320]:
pd.options.display.max_rows = 200
pd.options.display.max_columns = 200

In [321]:
df = pd.read_csv('data/listings_summary.csv')

# Explaining the goal:

My goal is to use NLP to, given a range of suggested prices for a property, determine if the host should select a value on the higher or lower end of that range.

The range of suggest prices is determined by using the XGBoost model used in the project, then plus and minus 20% of that value to get an upper and lower limit for suggested price. (20% is a little aibitrary. I found the MAE for predictions grouped by number of bedrooms. They were a little above 20%, but much higher gave too extreme of a range)

The suggestion for roughly where on that range the host should aim is determined by: 
- labeling the data by whether it is above or below the median market value for a unit with that number of bedrooms (number of bedrooms was selected because it had the highest feature importance in price prediction).
- tokenized and vectorized the descriptions of each unit, limiting myself to descriptions I'm fairly confident are mostly in English.
- Use logistic regression to predict probability of a description being associated with a price above median market value. (I didn't want to limit myself to exclusively English words, since certain German words might be used in mostly-English descriptions)
- Use that probability to give a general suggestion for where a user should aim in their price range

_the final results of the model are at the bottom of the notebook_

## Restricting to columns to use NLP with

In [322]:
df1 = df[['description','bedrooms','price']]

In [323]:
df1.head()

,description,bedrooms,price
0,Great location! 30 of 75 sq meters. This wood...,1.0,$60.00
1,In the summertime we are spending most of our ...,1.0,$17.00
2,This beautiful first floor apartment is situa...,1.0,$90.00
3,First of all: I prefer short-notice bookings. ...,1.0,$26.00
4,Cozy and large room in the beautiful district ...,1.0,$42.00


#### 'description' is the column we'll do NLP on, 'bedrooms' was determined in the exploratory notebook to have the highest association with price, and 'price' will be used to separate descriptions into 'high' and 'low' for nlp classification

## Cleaning Data

In [324]:
df1['price'] = df1['price'].str.replace('$','').str.replace(',','').astype(float)

In [325]:
df1 = df1.dropna()

In [326]:
# I was a bit more relaxed with the limits on this one
df1 = df1[(df1['price'] > 0) & (df1['price'] < 900)]

In [327]:
len(df1)

22281

In [328]:
# using common words to select only rows in English
df2 = df1[(df1['description'].str.contains(' the ')) |\
          (df1['description'].str.contains(' this ')) |\
          (df1['description'].str.contains(' and '))]

In [329]:
len(df2)

16169

In [330]:
df2.groupby(['bedrooms']).median()

,price
bedrooms,
0.0,50.0
1.0,40.0
2.0,80.0
3.0,120.0
4.0,150.0
5.0,240.0
6.0,349.0
7.0,460.0
8.0,584.0


In [331]:
df2_0 = df2[df2['bedrooms'] == 0]
df2_1 = df2[df2['bedrooms'] == 1]
df2_2 = df2[df2['bedrooms'] == 2]
df2_3 = df2[df2['bedrooms'] == 3]
df2_4 = df2[df2['bedrooms'] == 4]
df2_5 = df2[df2['bedrooms'] == 5]
df2_6 = df2[df2['bedrooms'] == 6]
df2_7 = df2[df2['bedrooms'] == 7]
df2_8 = df2[df2['bedrooms'] == 8]

In [332]:
df2_0['is_above_median'] = df2_0['price'].apply(lambda x: x >= df2_0['price'].median())
df2_1['is_above_median'] = df2_1['price'].apply(lambda x: x >= df2_1['price'].median())
df2_2['is_above_median'] = df2_2['price'].apply(lambda x: x >= df2_2['price'].median())
df2_3['is_above_median'] = df2_3['price'].apply(lambda x: x >= df2_3['price'].median())
df2_4['is_above_median'] = df2_4['price'].apply(lambda x: x >= df2_4['price'].median())
df2_5['is_above_median'] = df2_5['price'].apply(lambda x: x >= df2_5['price'].median())
df2_6['is_above_median'] = df2_6['price'].apply(lambda x: x >= df2_6['price'].median())
df2_7['is_above_median'] = df2_7['price'].apply(lambda x: x >= df2_7['price'].median())
df2_8['is_above_median'] = df2_8['price'].apply(lambda x: x >= df2_8['price'].median())

In [333]:
separated_frames = [df2_0,df2_1,df2_2,df2_3,df2_4,df2_5,df2_6,df2_7,df2_8]

In [334]:
df3 = pd.concat(separated_frames)

In [335]:
tf_counts = df3['is_above_median'].value_counts()

### Baseline to beat

In [336]:
tf_counts[1] / (tf_counts[1] + tf_counts[0])

0.5367060424268663

In [337]:
import re
# cleaning the description of non-alphanumeric values
df3['description_cleaned'] = df3['description'].apply(lambda x: re.sub(r'[^a-zA-Z ^0-9]', '', x))
# setting the string to lowercase, then splitting to create tokens
df3['tokens'] = df3['description_cleaned'].apply(lambda x: x.lower().split())

## Using tfidfVectorizer


In [338]:
# This sorts by reviews that are above or below median
df3 = df3.sort_values(by='is_above_median')

In [339]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [340]:
vectorizer = TfidfVectorizer(
    min_df=2,
    strip_accents = 'ascii'
)

In [341]:
tfidf = vectorizer.fit_transform(df3['description'])

In [342]:
# Pickling tfidf for app
import pickle
pickle.dump(tfidf, open('tfidf.pkl','wb'))

In [343]:
tfidf = tfidf.toarray()

In [344]:
tfidf.shape

(16169, 16692)

In [345]:
words = vectorizer.get_feature_names()

In [346]:
X = tfidf
y = df3['is_above_median'].astype(int)

In [347]:
from sklearn.model_selection import train_test_split

In [348]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=333)

In [349]:
from sklearn.linear_model import LogisticRegression

In [350]:
model = LogisticRegression().fit(X_train, y_train)

In [351]:
# Pickling logistic regression with word vectors for app
import pickle
pickle.dump(tfidf, open('nlp_lr_model.pkl','wb'))

In [352]:
model.predict_proba(X_test)

array([[0.56343433, 0.43656567],
       [0.6339595 , 0.3660405 ],
       [0.91804421, 0.08195579],
       ...,
       [0.57899581, 0.42100419],
       [0.38214468, 0.61785532],
       [0.20434294, 0.79565706]])

In [353]:
model.predict(X_test)

array([0, 0, 0, ..., 0, 1, 1])

In [354]:
model.score(X_test, y_test)

0.6950284442245857

## Exploring tokens associated with units above and below median price

In [355]:
import numpy as np

In [356]:
coef = model.coef_.reshape(-1)

In [357]:
np.argmax(coef)

9315

In [358]:
words[9369]

'loftbed'

In [359]:
high_idx = np.argsort(coef)[-10:]

In [360]:
high_idx

array([13498,  7435, 15225,  2499, 10046,  4604, 14681, 12426,  1610,
        9315], dtype=int64)

In [361]:
low_idx = np.argsort(coef)[:10]

In [362]:
low_idx

array([12601, 13246, 13245,  4620,  6103, 16047, 16168, 10507, 16621,
       12611], dtype=int64)

In [363]:
words = np.array(words)

In [364]:
words[high_idx]

array(['sleeping', 'heart', 'tv', 'berlin', 'mitte', 'design', 'terrace',
       'restaurants', 'apartment', 'living'], dtype='<U36')

In [365]:
words[low_idx]

array(['room', 'shared', 'share', 'desk', 'flatmate', 'wedding', 'wg',
       'neukolln', 'zimmer', 'roommate'], dtype='<U36')

## Testing model with user-generated description

In [366]:
example = ["This is a great living envorinment. We're located in the center of Berlin. Fully redisigned interior. Local restaurants exist right outside our door"]

In [367]:
example_transformed = vectorizer.transform(example)

In [368]:
model.predict_proba(example_transformed)

array([[0.19393825, 0.80606175]])

## Cleaning example of dataframe to test predictions

In [369]:
df_example = df.head(5)

In [370]:
df_example = df_example[['description','neighbourhood_group_cleansed','room_type','accommodates','bathrooms','bedrooms','beds',
          'bed_type','amenities','security_deposit','cleaning_fee', 'minimum_nights', 'price']]

In [371]:
df_example

,description,neighbourhood_group_cleansed,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,security_deposit,cleaning_fee,minimum_nights,price
0,Great location! 30 of 75 sq meters. This wood...,Mitte,Entire home/apt,3,1.0,1.0,2.0,Real Bed,"{TV,""Cable TV"",Wifi,Kitchen,Gym,Heating,""Famil...",$200.00,$30.00,4,$60.00
1,In the summertime we are spending most of our ...,Pankow,Private room,2,1.0,1.0,1.0,Real Bed,"{Wifi,Kitchen,Elevator,Heating,Washer,Essentia...",$0.00,$0.00,2,$17.00
2,This beautiful first floor apartment is situa...,Pankow,Entire home/apt,4,1.0,1.0,2.0,Real Bed,"{Internet,Wifi,Kitchen,""Buzzer/wireless interc...",$200.00,$50.00,62,$90.00
3,First of all: I prefer short-notice bookings. ...,Tempelhof - Schöneberg,Private room,2,1.0,1.0,1.0,Pull-out Sofa,"{Internet,Wifi,""Pets allowed"",""Pets live on th...",$250.00,$30.00,5,$26.00
4,Cozy and large room in the beautiful district ...,Pankow,Private room,2,1.0,1.0,2.0,Real Bed,"{Wifi,Heating,""Family/kid friendly"",Essentials...",$0.00,$0.00,2,$42.00


In [372]:
# cleaning dollar-amount columns
monetary_columns = ['security_deposit','cleaning_fee','price']
for item in monetary_columns:
    df_example[item] = df_example[item].str.replace('$','').str.replace(',','').astype(float)

In [373]:
used_amenities = ['Washer', 'Hair dryer', 'Laptop friendly workspace', 'Hangers',
       'Iron', 'Shampoo', 'TV', 'Hot water', 'Family/kid friendly', 'Internet',
       'Host greets you', 'Smoke detector', 'Buzzer/wireless intercom',
       'Lock on bedroom door', 'Free street parking', 'Elevator', 'Bed linens',
       'Smoking allowed', 'First aid kit', 'Cable TV']
for item in used_amenities:
    df_example[item] = df_example['amenities'].str.contains(item).astype(int)
df_example = df_example.drop(columns='amenities')

In [374]:
df_example

,description,neighbourhood_group_cleansed,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,security_deposit,cleaning_fee,minimum_nights,price,Washer,Hair dryer,Laptop friendly workspace,Hangers,Iron,Shampoo,TV,Hot water,Family/kid friendly,Internet,Host greets you,Smoke detector,Buzzer/wireless intercom,Lock on bedroom door,Free street parking,Elevator,Bed linens,Smoking allowed,First aid kit,Cable TV
0,Great location! 30 of 75 sq meters. This wood...,Mitte,Entire home/apt,3,1.0,1.0,2.0,Real Bed,200.0,30.0,4,60.0,0,1,1,1,1,1,1,1,1,0,0,1,0,1,0,0,1,0,0,1
1,In the summertime we are spending most of our ...,Pankow,Private room,2,1.0,1.0,1.0,Real Bed,0.0,0.0,2,17.0,1,1,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,This beautiful first floor apartment is situa...,Pankow,Entire home/apt,4,1.0,1.0,2.0,Real Bed,200.0,50.0,62,90.0,1,1,1,1,1,1,0,0,1,1,0,1,1,0,0,0,0,0,1,0
3,First of all: I prefer short-notice bookings. ...,Tempelhof - Schöneberg,Private room,2,1.0,1.0,1.0,Pull-out Sofa,250.0,30.0,5,26.0,1,1,1,1,1,1,0,1,0,1,1,1,1,0,1,0,1,0,0,0
4,Cozy and large room in the beautiful district ...,Pankow,Private room,2,1.0,1.0,2.0,Real Bed,0.0,0.0,2,42.0,0,1,1,1,0,1,0,1,1,0,0,0,0,0,0,0,1,0,0,0


# First predicting the price range, then offering a recommendation on where to select in the range based on the NLP prediction

In [375]:
import pickle

In [376]:
price_model = pickle.load(open('model.pkl', 'rb'))

In [377]:
# Dropping columns to use with price prediction model
df_price_prediction = df_example.drop(columns=['description','price'])

In [378]:
price_predictions = price_model.predict(df_price_prediction)

In [379]:
price_predictions

array([76.14437 , 36.75857 , 74.26539 , 37.7499  , 38.892452],
      dtype=float32)

In [380]:
df_above_median = df_example[['description','bedrooms']]
df_above_median['price'] = price_predictions

In [381]:
df_example_description_transformed = vectorizer.transform(df_example['description'])
predict_probas = model.predict_proba(df_example_description_transformed)

In [382]:
predict_probas

array([[0.3407018 , 0.6592982 ],
       [0.70646548, 0.29353452],
       [0.21735763, 0.78264237],
       [0.62607432, 0.37392568],
       [0.6621181 , 0.3378819 ]])

In [383]:
predict_probas[:, 1]

array([0.6592982 , 0.29353452, 0.78264237, 0.37392568, 0.3378819 ])

In [384]:
d = {'predicted_prices': price_predictions, 'probability_above_median': predict_probas}

In [385]:
predictions_df = df_price_prediction.copy()

In [386]:
predictions_df['predicted_prices'] = price_predictions
predictions_df['probability_above_median'] = predict_probas[:, 1]

In [387]:
preds_df = predictions_df[['predicted_prices','probability_above_median']]

In [388]:
preds_df['price_range_low'] = preds_df['predicted_prices'].apply(lambda x: x * .8)
preds_df['price_range_high'] = preds_df['predicted_prices'].apply(lambda x: x * 1.2)

In [389]:
preds_df

,predicted_prices,probability_above_median,price_range_low,price_range_high
0,76.144371,0.659298,60.915497,91.373245
1,36.758572,0.293535,29.406857,44.110286
2,74.265388,0.782642,59.412311,89.118466
3,37.749901,0.373926,30.199921,45.299881
4,38.892452,0.337882,31.113962,46.670943


In [390]:
def price_range_recommendation(df):
    upper_range = int(df['price_range_high'])
    lower_range = int(df['price_range_low'])
    proba = df['probability_above_median']
    if proba < 0.20:
        message = "the low end of that range."
    elif proba >= 0.20 and proba < 0.4:
        message = "the medium-low end of that range."
    elif proba >=0.4 and proba < 0.6:
        message = "about the middle of that range."
    elif proba >=0.6 and proba < 0.8:
        message = "the medium-high end of that range."
    elif proba >= 0.8:
        message = "the high end of the range"
    print(f'Based on your inputs, the model recommends your unit cost between ${lower_range} and ${upper_range}.')
    print(f'Based on your text discription, the model recommends you select a price at {message}')
    

In [391]:
price_range_recommendation(preds_df.iloc[0])

Based on your inputs, the model recommends your unit cost between $60 and $91.
Based on your text discription, the model recommends you select a price at the medium-high end of that range.


In [393]:
for num in range(5):
    price_range_recommendation(preds_df.iloc[num])
    print(" ")

Based on your inputs, the model recommends your unit cost between $60 and $91.
Based on your text discription, the model recommends you select a price at the medium-high end of that range.
 
Based on your inputs, the model recommends your unit cost between $29 and $44.
Based on your text discription, the model recommends you select a price at the medium-low end of that range.
 
Based on your inputs, the model recommends your unit cost between $59 and $89.
Based on your text discription, the model recommends you select a price at the medium-high end of that range.
 
Based on your inputs, the model recommends your unit cost between $30 and $45.
Based on your text discription, the model recommends you select a price at the medium-low end of that range.
 
Based on your inputs, the model recommends your unit cost between $31 and $46.
Based on your text discription, the model recommends you select a price at the medium-low end of that range.
 
